# Module 2 - Lab 2 - Building a Recommendation Engine in Spark

In this lab, we will build a recommendation model in Spark. 

The idea behind recommendation engines is to predict what people might like and to uncover relationships between items to aid in the discovery process. They are similar to search engines. However, unlike search engines they try to present individuals with relevant content that they did not necessarily search for or that they might not have heard of. 

Usually, a recommendation engine tries to model the connections between users and some type of item. They are most effective in two general scenarios:
* Large number of available options for users: When there is a very large number of available items, it becomes increasingly difficult for the user to find something they want. 
* A significant degree of personal taste involved: When personal taste plays a large role in selection, recommendation models, which often utilize a wisdom of the crowd approach, can be helpful in discovering items based on the behavior of others that have similar taste profiles. 

Amazon's shopping cart recommendations are a good example. So are Netflix movie recommendations. 

There are many types of recommendation models. The two that are most prevalent are: content-based filtering and collaborative filtering. Content-based filtering attemps to use the content or attributes of an item, together with some notion of similarity between two pieces of content to generate items similar to a given item. Collaborative filtering is a form of wisdom of the crowd approach where the set of preferences of users for items with which they have not yet interacted. The underlying idea is the notion of similarity. 

The code snipet below initializes our pyspark kernel. 

In [1]:
import org.apache.spark.mllib.recommendation.ALS
import org.apache.spark.mllib.recommendation.Rating
import org.jblas.DoubleMatrix
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.mllib.evaluation.RankingMetrics

println("Spark Initialized")

Creating SparkContext as 'sc'


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
155,None,spark,idle,,,✔


Creating HiveContext as 'sqlContext'
SparkContext and HiveContext created. Executing user code ...
Spark Initialized

# Extracting features from the MovieLens 100k dataset

In this example, we are using the MovieLens dataset. Remember that the dataset consisted of the 
 - user ID, 
 - movie ID, 
 - rating, and 
 - timestamp 
 
fields, all separated by the tab character. We don't need the time when the rating was given to train the model, so we will only extract the first three fields after inspecting the raw dataset. 

In [2]:
val rawData = sc.textFile("ml-100k/u.data")
rawData.first()

res2: String = 196	242	3	881250949

In [3]:
val rawRatings = rawData.map(_.split("\t").take(3))
rawRatings.first()

res3: Array[String] = Array(196, 242, 3)

In [4]:
val ratings = rawRatings.map {case Array(user, movie, rating) => Rating(user.toInt, movie.toInt, rating.toDouble)}
ratings.first()

res4: org.apache.spark.mllib.recommendation.Rating = Rating(196,242,3.0)

**This is a map, then, of the ratings people made on individual movies, extracted as "features". A **feature** is simply a way of expressing something that's useful to us. How people rated movies can then be used to predict how other people might like movies. **

# Training the recommendation model

Once we have extracted these simple features from our raw data, we are ready to proceed with the model training. We need to provide the correctly-parsed input RDD we just creaetd as well as our chosen model parameters. 

Let's train the model! The inputs required for the model are as follows:
* rank: This refers to the number of factors in the model. Generally, the greater the number of factors, the better, but this has a direct impact on memory usage, both for computation and to store models for serving, particularly for large number of users or items. 
* iterations: This refers to the number of iterations to run. While each iteration is guaranteed to decrease the reconstruction error of the ratings matrix, our model should converge to a reasonable solution after relatively few iterations. Therefore, we don't need to run for too many iterations in most cases.
* lambda: This parameter controls the regularizaion of our model. Thus, lamba controls overfitting. The higher the value of lambda, the more the regularization is applied. What constitutes a sensible value is very dependent on the size, nature, nad sparsity of the underlying data, and as with almost all machine learning models, the regularization parameter is somthing that should be tuned using out-of-sample test data and cross-validation approaches.

We are using a rank of 50, a lambda value of 0.01, and 10 iterations to train our model.

This returns a MatrixFactorizationModel object, which contains the user and item factors in the form of an RDD pair. These are called userFeatures and productFeatures.

In [5]:
val model = ALS.train(ratings, 50, 10, 0.01)
model.userFeatures

res5: org.apache.spark.rdd.RDD[(Int, Array[Double])] = users MapPartitionsRDD[209] at mapValues at ALS.scala:255

We can force the computation using the count Spark action.

In [6]:
model.userFeatures.count
model.productFeatures.count

res7: Long = 1682

# Using the recommendation model

Now that we have trained the model, we can use it to make preditions. These predictions typically take one of two forms: recommendations for a given user and related or similar items for a given item.

** This is how collaborative filtering works : In effect, the recommendation model will search for users who rated some films like you did. Then it will find movies that you have not rated, but that these other users who rated some of the same films you did, like ** 

In our case, we want to generate recommended items (*movies*) for a given user. This will take the form of a top-K list, which means that the K items that our model predicts will have the highest probability of hte user liking them. We do this by predicting the scores for each item (*movie*) and ranking the list based on the score. The exact method to perform this computation depends on the model involved. For now, we are sheilding you from these internal details. 

## Generating recommendations

We will use the factor matrices computed by our model to compute the predicted scores (or ratings) for a user. We will focus on the explicit rating case using MovieLens data. However, the approach is the same when using the implicit model. 

The predict() method below is used to compute a predict a score for a given user and item (movie) combination.

In [43]:
val predictRating = model.predict(789, 123)

predictRating: Double = 3.8825736731923217

To generate the top-K recommended items for a user, MatrixFactorizationModel provides a convenience method called recommendProducts(). This takes in two arguments: user and num, where user is the user ID, and num is the number of items to recommend. It returns the top num items ranked in the order of the predicted score. 

** This is pretty cool. Its how product recommendations work on major websites**

In [44]:
val userId = 789
val K = 10
val topKRecs = model.recommendProducts(userId, K)

println(topKRecs.mkString("\n"))

Rating(789,182,5.9027642432595)
Rating(789,317,5.731498054760541)
Rating(789,156,5.687498683675839)
Rating(789,192,5.598312888241889)
Rating(789,56,5.5976131209991395)
Rating(789,87,5.545339018005654)
Rating(789,58,5.537538780943374)
Rating(789,641,5.438618026363299)
Rating(789,530,5.391429476559729)
Rating(789,157,5.362021180629817)

## Inspecting Recommendations

We can give these recommendations a sense check by taking a quick look at the titles of hte movies a user has rated and hte recommended movies. Firstly, we have the load the movie data. We will gather the data as a Map[Int, String] method mapping the movie ID to the title.

In [45]:
val movies = sc.textFile("ml-100k/u.item")
val titles = movies.map(line => line.split("\\|").take(2)).map(array => (array(0).toInt, array(1))).collectAsMap()
titles(123)

res36: String = Frighteners, The (1996)

Here we observe the movie ratings for user 789. 

In [46]:
val moviesForUser = ratings.keyBy(_.user).lookup(789)
println(moviesForUser.size)

33

Here we take the 10 movies with the highest ratings by sorting the moviesForUser collection using the rating field of the Rating object. 

In [47]:
moviesForUser.sortBy(-_.rating).take(10).map(rating => (titles(rating.product), rating.rating)).foreach(println)

(Godfather, The (1972),5.0)
(Trainspotting (1996),5.0)
(Dead Man Walking (1995),5.0)
(Star Wars (1977),5.0)
(Swingers (1996),5.0)
(Leaving Las Vegas (1995),5.0)
(Bound (1996),5.0)
(Fargo (1996),5.0)
(Last Supper, The (1995),5.0)
(Private Parts (1997),4.0)

Here we take the top 10 recommendations for the user and see what the titles are using the same approach as the one we used earlier.

In [48]:
topKRecs.map(rating => (titles(rating.product), rating.rating)).foreach(println)

(GoodFellas (1990),5.9027642432595)
(In the Name of the Father (1993),5.731498054760541)
(Reservoir Dogs (1992),5.687498683675839)
(Raging Bull (1980),5.598312888241889)
(Pulp Fiction (1994),5.5976131209991395)
(Searching for Bobby Fischer (1993),5.545339018005654)
(Quiz Show (1994),5.537538780943374)
(Paths of Glory (1957),5.438618026363299)
(Man Who Would Be King, The (1975),5.391429476559729)
(Platoon (1986),5.362021180629817)

# Generating similar movies

Here, we will create our own code to generate similar movies. We will use the cosine similarity metric and the jblas linear algebra library to compute the required vector dot products. 

**WHAT?!** : We are looking for movies that are similar to each other.

Then, we will compare the factor vector of our chosen item with each of the other items, using our similarity metric. Firstly, we create a vector object out of the factor vectors, which are in the form of an Array[Double].

In [49]:
val aMatrix = new DoubleMatrix(Array(1.0, 2.0, 3.0))

aMatrix: org.jblas.DoubleMatrix = [1.000000; 2.000000; 3.000000]

Next, we create a method to compute the cosine similarity between two vectors. Cosine similarity is a measure of the angle between two vectors in an n-dimensional space. We compute this value by first calculating the dot product between the vectors and then dividing the result by a denominator, which is the norm (or length) of each vector multiplied together. In this way, cosine similarity is a normalized dot product. 

The cosine similarity measure takes on values between -1 and 1. a value of 1 implies a completely similar status, while a value of 0 implies independence. We can also capture the negative similarity, that is a value of -1 implies that not only are the vectors not similar, but they are also completely dissimilar.

In [50]:
def cosineSimilarity(vec1: DoubleMatrix, vec2: DoubleMatrix): Double =
{
    vec1.dot(vec2) / (vec1.norm2() * vec2.norm2())
}

cosineSimilarity: (vec1: org.jblas.DoubleMatrix, vec2: org.jblas.DoubleMatrix)Double

Here, we will try out our function on item 567. We will collect an item factor form out model using the lookup() method. We use the head() function to obtain the first value. We create the DoubleMatrix object from the created Array[Double].

In [51]:
val itemId = 567
val itemFactor = model.productFeatures.lookup(itemId).head
val itemVector = new DoubleMatrix(itemFactor)
cosineSimilarity(itemVector, itemVector)

res40: Double = 0.9999999999999998

A similarity metric should measure how close two vectors are to each other. Above, we can see that our metric shows that the item vector is idetical to itself. Below, we apply our similarity metric to each item. 

In [52]:
val sims = model.productFeatures.map{ case (id, factor) =>
                                    val factorVector = new DoubleMatrix(factor)
                                    val sim = cosineSimilarity(factorVector, itemVector)
                                    (id, sim)
                                    }

sims: org.apache.spark.rdd.RDD[(Int, Double)] = MapPartitionsRDD[469] at map at <console>:54

Below, we compute the top 10 most similar items by sorting out the similarity score for each item. We use the top() function, which computes the top-K results in a distributed fashion instead of using collect9) to return all the data to the driver and sorting it locally. Here, we tell Spark how to sort the paris in the sims RDD. To do this, we pass the Ordering object, which tells Spark that it should sort by the value in the key-value pair (a.k.a. sort by similarity). We then print the 10 items with the highest computed similarity metric to our given item.

In [53]:
val sortedSims = sims.top(K)(Ordering.by[(Int, Double), Double] { case (id, similarity) => similarity })
println(sortedSims.take(10).mkString("\n"))

(567,0.9999999999999998)
(1222,0.7208308624354276)
(184,0.7015866096597198)
(1002,0.7000642613423828)
(271,0.6862652733768038)
(550,0.6824989059373686)
(152,0.6810681685408917)
(219,0.6772090230791078)
(448,0.6755880103107036)
(563,0.6738665950586353)

## Inspecting similar items

Below, we observe the title of our chosen movie. 

In [54]:
println(titles(itemId))

Wes Craven's New Nightmare (1994)

Here, we look at the titles of the most similar movies. We take the top 11 so that we can exclude our given movie.

In [55]:
val sortedSims2 = sims.top(K + 1)(Ordering.by[(Int, Double), Double] { case (id, similarity) => similarity })
sortedSims2.slice(1,11).map{ case (id, sim) => (titles(id), sim)}.mkString("\n")

res43: String = 
(Judgment Night (1993),0.7208308624354276)
(Army of Darkness (1993),0.7015866096597198)
(Pest, The (1997),0.7000642613423828)
(Starship Troopers (1997),0.6862652733768038)
(Die Hard: With a Vengeance (1995),0.6824989059373686)
(Sleeper (1973),0.6810681685408917)
(Nightmare on Elm Street, A (1984),0.6772090230791078)
(Omen, The (1976),0.6755880103107036)
(Stephen King's The Langoliers (1995),0.6738665950586353)
(Men in Black (1997),0.6730776169626559)

# Evaluating Performance

In order to detect whether the model we have trained is a good model or not, we need to be able to evaluate its predictive performance in some way. Evalution metrics are measures of a model's predictive capability or accuracy. Some are direct measures of how well a model predicts the model's target variable while others are concerned with how well the model performs at predicting things that mights not be directly optimized in the model but are often closer to what we care about in the real world. 

These metrics provide a standardized way of comparing the performance of the same model with different parameter setting and of comparing performance across different models. Using these metrics, we can perform model selection to choose the best-performing model from the set of models we wish to evalute.

Below, we will calculate two common evaluation metrics used in reommender systems and collaborative filtering models: Mean Squared Error and Mean average precision at K.

## Mean Squared Error

Mean Squared Error (MSE) is a direct measure of the reconstruction error of the user-item rating matrix. It is also the objective function being minimized in certain models, specifically many matrix-factorization techniques. Therefore, it is commonly used in explicit ratings settings. 

It is defined as the sum of the squared errors divided by the number of observations. The squared error, in turn, is the square of the difference between the predicted rating for a given user-item pair and the actual rating. 

Here, we take the first rating for user 789 from the moviesForUser set of Ratings previously computed.

In [56]:
val actualRating = moviesForUser.take(1)(0)

actualRating: org.apache.spark.mllib.recommendation.Rating = Rating(789,1012,4.0)

We now compute the model's predicted rating. 

In [57]:
val predictedRating = model.predict(789, actualRating.product)

predictedRating: Double = 4.074667240270278

We now compute the squred error between the actual rating and the predicted rating.

In [58]:
val squaredError = math.pow(predictedRating - actualRating.rating, 2.0)

squaredError: Double = 0.005575196769579448

In order to compute the overall MSE for the dataset, we need to compute the squared error for each entry, sum them up, and divide them by the number of ratings.

Firstly, we extract the user and product IDs from the ratings RDD and make predictions for each user-item pair using model.predict(). We will then use the user-item pair as the key and the predicted rating as the value.

In [59]:
val usersProducts = ratings.map{ case Rating(user, product, rating) => (user, product)}
val predictions = model.predict(usersProducts).map{
    case Rating(user, product, rating) => ((user, product), rating)
}

predictions: org.apache.spark.rdd.RDD[((Int, Int), Double)] = MapPartitionsRDD[481] at map at <console>:48

Next, we extract the actual ratings and also map the ratings RDD so that the user-item pair is the key and the actual rating is the value. We can now join the two RDDs together to create a new RDD with the actual and predicted ratings for each user-item combination.

In [60]:
val ratingsAndPredictions = ratings.map{
    case Rating(user, product, rating) => ((user, product), rating)
}.join(predictions)

ratingsAndPredictions: org.apache.spark.rdd.RDD[((Int, Int), (Double, Double))] = MapPartitionsRDD[485] at join at <console>:52

Now, we compute the MSE by summing up the squared errors using reduce and dividing by the count method of the number of records.

In [61]:
val MSE = ratingsAndPredictions.map{
    case ((user, product), (actual, predicted)) => math.pow((actual - predicted), 2)
}.reduce(_ + _) / ratingsAndPredictions.count
println("Mean Suared Error = " + MSE)

Mean Suared Error = 0.08470571763977897

It is common to use the Root Mean Squared Error (RMSE), which is the square root of the MSE metric. This is somewhat more interpretable, since it is the same units as the underlying data. It is equivalent to the standard deviation of the differences between the predicted and actual ratings. 

In [62]:
val RMSE = math.sqrt(MSE)
println("Root Mean Squared Error = " + RMSE)

Root Mean Squared Error = 0.2910424670727263

## Mean Average Precision at K

Mean average precision at K (MAPK) is the mean of the average precision at K (APK) metric across all instances in the dataset. APK is a metric commonly used in information retrieval. APK is a measure of the average relevance scores of a set of the top-K documents presented in response to a query. For every query instance, we compare the set of top-K results with the set of actual relevant documents.

In the APK metric, the order of the result set matters in that the APK score would be higher if the result documents are both relevant nad the relevant documents are presented higher in the results. Therefore, it is a solid metric for recommender systems in that typically we would compute the top-K recommended items for each user and present these to the user. APK and other ranking-based metrics are also more appropriate evaluation measures for implicit datasets. Here, MSE makes less sense. 

In order to evaluate our model, we use APK, where each user is the equivalent of a query, and the set of top-K recommended items is the document result set. The relevant documents in our case, is the set of items that a user interacted with. Therefore, APK attempts to measure how good our model is at predicting items that a user will find relevant and choose to interact with. 

Here is our fucntion to compute the APK.

In [63]:
def avgPrecisionK(actual: Seq[Int], predicted: Seq[Int], k: Int):
Double = {
    val predK = predicted.take(k)
    var score = 0.0
    var numHits = 0.0
    for ((p, i) <- predK.zipWithIndex) {
        if (actual.contains(p)) {
            numHits += 1.0
            score += numHits / (i.toDouble + 1.0)
        }
    }
    if (actual.isEmpty) {
        1.0
    } else {
        score / scala.math.min(actual.size, k).toDouble
    }
}

avgPrecisionK: (actual: Seq[Int], predicted: Seq[Int], k: Int)Double

Here we compute the APK metric for example user 789. 

First we extract the actual movie IDs for the user.

In [64]:
val actualMovies = moviesForUser.map(_.product)

actualMovies: Seq[Int] = ArrayBuffer(1012, 127, 475, 93, 1161, 286, 293, 9, 50, 294, 181, 1, 1008, 508, 284, 1017, 137, 111, 742, 248, 249, 1007, 591, 150, 276, 151, 129, 100, 741, 288, 762, 628, 124)

Then, we use the movie recommendations we made previously to compute the APK score.

In [65]:
val predictedMovies = topKRecs.map(_.product)

predictedMovies: Array[Int] = Array(182, 317, 156, 192, 56, 87, 58, 641, 530, 157)

The following code will produce the average precision.

In [66]:
val apk10 = avgPrecisionK(actualMovies, predictedMovies, 10)

apk10: Double = 0.0

In order to compute the APK for each user and average them to compute the overall MAPK, we need to generate the list of recommendations for each user in our dataset. While this can be fairly intensive on a large scale, we can distribute the computation using our Spark functionality. However, one limitation is that each worder must have the full item-factor matrix available so that it can compute the dot product between the relevant user matrix available so that it can compute the dot product between the relevant user vector and all item vectors. This can be a problem when the number of items is extremely high as the item matrix must fit in the memory of one machine. 

To do this, we first collect the item factors and form a DoubleMatrix object from them.

In [67]:
val itemFactors = model.productFeatures.map { case (id, factor) => factor }.collect()
val itemMatrix = new DoubleMatrix(itemFactors)
println(itemMatrix.rows, itemMatrix.columns)

(1682,50)

Next, we distribute the item matrix as a broadcast variable so that it is available on each worker node in the cluster.

In [68]:
val imBroadcast = sc.broadcast(itemMatrix)

imBroadcast: org.apache.spark.broadcast.Broadcast[org.jblas.DoubleMatrix] = Broadcast(121)

Now we compute the recommendations for each user. We do this by applying a map function to each user factor within which we will perform a matrix multiplication between the user-factor vector and the movie-factor matrix. The result is a vector with the predicted rating for each movie. We then sort these predictions by the predicted rating.

In [69]:
val allRecs = model.userFeatures.map{ case (userId, array) =>
    val userVector = new DoubleMatrix(array)
    val scores = imBroadcast.value.mmul(userVector)
    val sortedWithId = scores.data.zipWithIndex.sortBy(-_._1)
    val recommendedIds = sortedWithId.zipWithIndex.map(_._2 + 1).toSeq
    (userId, recommendedIds)
}

allRecs: org.apache.spark.rdd.RDD[(Int, Seq[Int])] = MapPartitionsRDD[488] at map at <console>:54

We also need a list of movie IDs for each user to pass into our APK fucntion as the actual argument. We already have the ratings RDD ready, so we can extract the user nad movie IDs from it. 

In [70]:
val userMovies = ratings.map{ case Rating(user, product, rating) => (user, product) }.groupBy(_._1)

userMovies: org.apache.spark.rdd.RDD[(Int, Iterable[(Int, Int)])] = ShuffledRDD[491] at groupBy at <console>:44

Finally, we can use Sparks join() operator to join these two RDDs together on the user ID key. Then, for every user, we have the list of actual and predicted movie IDs that we can pass to our APK function. In a monner similar to how we computed MSE, we will sum each of these APK scores using a reduce action and divide by the number of users.

In [71]:
val K = 10
val MAPK = allRecs.join(userMovies).map{ case (userId, (predicted, actualWithIds)) =>
    val actual = actualWithIds.map(_._2).toSeq
    avgPrecisionK(actual, predicted, K)
}.reduce(_ + _) / allRecs.count
println("Mean Average Precision at K = " + MAPK)

Mean Average Precision at K = 0.1300756619367436

# Using MLlib

Here, we are using MLlib, which provides convenience functions to compute MSE, RMSE, and MAPK in the RegressionMetrics and RankingMetrics classes.

First, we compute the MSE and RMSE metrics using RegressionMetrics. We instantiate the predicted and true values for each data point. We again use the ratingsAndPredictions RDD we computed in the previous example. 

In [72]:
val predictedAndTrue = ratingsAndPredictions.map { case ((user, product), (predicted, actual)) => (predicted, actual) }
val regressionMetrics = new RegressionMetrics(predictedAndTrue)

println("Mean Squared Error = " + regressionMetrics.meanSquaredError +
        "\nRoot Mean Squared Error = " + regressionMetrics.rootMeanSquaredError)

Mean Squared Error = 0.08470571763977897
Root Mean Squared Error = 0.2910424670727263

We can compute ranking-based evaluation metrics using MLlib's RankingMetrics class. Similarly, to our own average precision function, we need to pass in an RDD of key-value pairs, where the key is an Array of predicted item IDs for a user, while the value is an array of actual item IDs.

First, we calculate MAP using RankingMetrics().

In [73]:
val predictedAndTrueForRanking = allRecs.join(userMovies).map{ case (userId, (predicted, actualWithIds)) =>
    val actual = actualWithIds.map(_._2)
    (predicted.toArray, actual.toArray)
}
val rankingMetrics = new RankingMetrics(predictedAndTrueForRanking)
println("Mean Average Precision = " + rankingMetrics.meanAveragePrecision)

Mean Average Precision = 0.15884678068634858

Next, we use our function to compute the MAP in exactly the same way as we did previously, except that we set K to a very high value (i.e. 2000).

In [74]:
val MAPK2000 = allRecs.join(userMovies).map{ case (userId, (predicted, actualWithIds)) => 
    val actual = actualWithIds.map(_._2).toSeq
    avgPrecisionK(actual, predicted, 2000)
}.reduce(_ + _) / allRecs.count
println("Mean Average Precision = " + MAPK2000)

Mean Average Precision = 0.15884678068634836

How good is this [mean average precision](http://fastml.com/what-you-wanted-to-know-about-mean-average-precision/)? Think about it in the context of how diverse movies and ratings can be, and how often movie recommendations work out for you!